In [1]:
import pandas as pd
from openai import OpenAI
client = OpenAI()

# 1. Prepare input file

In [2]:
# Define the relative path to the CSV file
file_path = "../source_data/olist_order_reviews_dataset.csv"

# Load CSV file into a pandas DataFrame
df = pd.read_csv(file_path)

# Display the first few rows to confirm
display(df.head())

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


# 2. Uploading the file

In [4]:
batch_input_file = client.files.create(
    file=open("batch_test.jsonl", "rb"),
    purpose="batch"
)
file_id = batch_input_file.id
print(batch_input_file)

FileObject(id='file-RyAWj53mDbf3rX4wi5a8N2', bytes=516, created_at=1736594196, filename='batch_test.jsonl', object='file', purpose='batch', status='processed', status_details=None)


# 3. Creating the batch

In [14]:
batch_input_file_id = batch_input_file.id
batch_response = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "Airflow Demo Test Job"
    }
)

batch_id = batch_response.id

# 4. Checking the Status of a Batch

In [25]:
batch = client.batches.retrieve(batch_id)
output_file_id = batch.output_file_id
print(batch.status)
print(output_file_id)

completed
file-XFjAVjTCkhTygULJUSSfVp


# 5. Retrieving the results

In [26]:
file_response = client.files.content(output_file_id)
print(file_response.text)

{"id": "batch_req_6782553e4a688190bf3bb781cfb3e0c1", "custom_id": "request-1", "response": {"status_code": 200, "request_id": "5c73d3b3071870b291bcb8969fef4827", "body": {"id": "chatcmpl-AoTufo7r50w9JL09btx1LHiRL58VJ", "object": "chat.completion", "created": 1736594749, "model": "gpt-3.5-turbo-0125", "choices": [{"index": 0, "message": {"role": "assistant", "content": "Hello! How can I assist you today?", "refusal": null}, "logprobs": null, "finish_reason": "stop"}], "usage": {"prompt_tokens": 20, "completion_tokens": 10, "total_tokens": 30, "prompt_tokens_details": {"cached_tokens": 0, "audio_tokens": 0}, "completion_tokens_details": {"reasoning_tokens": 0, "audio_tokens": 0, "accepted_prediction_tokens": 0, "rejected_prediction_tokens": 0}}, "service_tier": "default", "system_fingerprint": null}}, "error": null}
{"id": "batch_req_6782553e69188190a9b33d7b9e4c4318", "custom_id": "request-2", "response": {"status_code": 200, "request_id": "0dc1682b335e9562a71701534f60cefc", "body": {"id

# 6. Cancelling results

In [ ]:
client.batches.cancel("batch_abc123")
# latest batch batch_678736d7ec408190a2b7620c8d94ad8a



# Custom

In [6]:
batch

Batch(id='batch_678736d7ec408190a2b7620c8d94ad8a', completion_window='24h', created_at=1736914648, endpoint='/v1/chat/completions', input_file_id='file-NmXzviEdfQCbkU5NXFnQsC', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1736925921, error_file_id=None, errors=None, expired_at=None, expires_at=1737001048, failed_at=None, finalizing_at=1736921059, in_progress_at=1736914657, metadata=None, output_file_id='file-F8A3C9cRNoiTwu3FKLTSs8', request_counts=BatchRequestCounts(completed=40572, failed=0, total=40572))

In [2]:
batch = client.batches.retrieve('batch_678736d7ec408190a2b7620c8d94ad8a')
output_file_id = batch.output_file_id
print(batch.status)
print(output_file_id)

completed
file-F8A3C9cRNoiTwu3FKLTSs8


In [5]:
file_response = client.files.content(output_file_id)
local_file = f"translations_output_custom.jsonl"
with open(local_file, "w") as f:
    f.write(file_response.text)